<a href="https://colab.research.google.com/github/Monocyte572/autodock-vina-Error-/blob/main/autodock.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# smiles to pdbqt

In [ ]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import MolToPDBBlock
import os
import traceback

csv_path = "/content/train.csv"
output_folder = "/content/output_folder"

df = pd.read_csv(csv_path)

os.makedirs(output_folder, exist_ok=True)

log_file = os.path.join(output_folder, "error_log.txt")

success_df = pd.DataFrame(columns=df.columns)

with open(log_file, "w") as log:
    for index, row in df.iterrows():
        smiles = row["Canonical_Smiles"]
        compound_name = row["ID"]

        try:
            if pd.isna(smiles) or not Chem.MolFromSmiles(smiles):
                raise ValueError(f"Invalid SMILES in row {index + 2}")

            mol = Chem.MolFromSmiles(smiles)
            mol = Chem.AddHs(mol)
            AllChem.EmbedMolecule(mol)

            pdbqt_filename = os.path.join(output_folder, f"{compound_name}.pdbqt")
            with open(pdbqt_filename, "w") as pdbqt_file:
                pdbqt_file.write(MolToPDBBlock(mol))


            success_df = success_df.append(row, ignore_index=True)

        except Exception as e:
            log.write(f"Error processing row {index + 2}: {str(e)}\n")
            traceback.print_exc(file=log)


success_df.to_csv(csv_path, index=False)

print("Conversion complete. Check the error log for details.")


Conversion complete. Check the error log for details.


In [ ]:
!pip install scripts.prepare_receptor

ERROR: Could not find a version that satisfies the requirement scripts.prepare_receptor (from versions: none)
ERROR: No matching distribution found for scripts.prepare_receptor


# autodock vina

In [27]:
import os, glob, subprocess
import pandas as pd
from tqdm import tqdm

# --- 설정 ---
receptor     = "/content/receptor.pdbqt"
ligand_dir   = "/content/output_folder"
output_dir   = "/content/out_path"
os.makedirs(output_dir, exist_ok=True)

# Grid box 설정 (예시값)
center_x, center_y, center_z = 22.57, 22.77, 11.81
size_x,   size_y,   size_z   = 5.58,  13.38,  10.50
exhaustiveness = 8

# --- 도킹 반복 실행 ---
results = []
for ligand_path in tqdm(glob.glob(os.path.join(ligand_dir, "*.pdbqt"))):
    ligand_name = os.path.basename(ligand_path)
    out_pdbqt   = os.path.join(output_dir, ligand_name.replace(".pdbqt", "_out.pdbqt"))
    log_file    = os.path.join(output_dir, ligand_name.replace(".pdbqt", ".log"))

    cmd = [
        "vina",
        "--receptor", receptor,
        "--ligand",   ligand_path,
        "--out",      out_pdbqt,
        "--log",      log_file,
        "--center_x", str(center_x),
        "--center_y", str(center_y),
        "--center_z", str(center_z),
        "--size_x",   str(size_x),
        "--size_y",   str(size_y),
        "--size_z",   str(size_z),
        "--exhaustiveness", str(exhaustiveness),
    ]
    subprocess.run(cmd, check=True)

    # 첫 번째 포즈(라인 시작이 "1")에서 affinity 추출
    affinity = None
    with open(log_file) as f:
        for line in f:
            if line.strip().startswith("1"):
                affinity = float(line.split()[1])
                break

    results.append({"ligand": ligand_name, "affinity_kcal_per_mol": affinity})

# --- 결과 CSV로 저장 ---
df = pd.DataFrame(results)
csv_path = os.path.join(output_dir, "docking_results.csv")
df.to_csv(csv_path, index=False)

print("✅ 도킹 완료! 결과는", csv_path, "에 저장되었습니다.")

  0%|          | 0/1681 [00:00<?, ?it/s]


CalledProcessError: Command '['vina', '--receptor', '/content/receptor.pdbqt', '--ligand', '/content/output_folder/TRAIN_1423.pdbqt', '--out', '/content/out_path/TRAIN_1423_out.pdbqt', '--log', '/content/out_path/TRAIN_1423.log', '--center_x', '22.57', '--center_y', '22.77', '--center_z', '11.81', '--size_x', '5.58', '--size_y', '13.38', '--size_z', '10.5', '--exhaustiveness', '8']' returned non-zero exit status 1.

In [25]:
!apt-get update
!apt-get install -y autodock-vina


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading